# Pytorch 学习笔记

## Tensor 张量
Tensor(张量）类似于NumPy的ndarray，但可以在GPU上使用来加速计算。

In [4]:
import torch
x = torch.empty(5, 3) # 未初始化的矩阵
print(x)
x = torch.rand(5, 3) # 0~1均匀分布的随机矩阵
print(x)
x = torch.zeros(5, 3, dtype=torch.long) # 零矩阵
print(x)
x = torch.tensor([5.5, 3]) # 自定义矩阵
print(x)
print(x.size()) # 注意：此时x为一维矩阵，与二维不同

tensor([[9.2755e-39, 1.0561e-38, 5.9694e-39],
        [7.8061e-39, 8.4490e-39, 9.6428e-39],
        [1.1112e-38, 9.5511e-39, 1.0102e-38],
        [1.0286e-38, 1.0194e-38, 9.6429e-39],
        [9.2755e-39, 9.1837e-39, 9.3674e-39]])
tensor([[1.8161e-01, 7.8112e-01, 6.0695e-04],
        [3.6412e-01, 3.9154e-01, 8.4160e-01],
        [2.4041e-02, 5.1627e-02, 4.4496e-01],
        [5.4531e-02, 9.9299e-01, 5.3873e-01],
        [3.0929e-01, 8.4750e-01, 8.6065e-01]])
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])
tensor([5.5000, 3.0000])
torch.Size([2])


## 运算

In [5]:
x = torch.rand(5, 3)
y = torch.rand(5, 3)
# 一
z = x + y
# 二
z = torch.add(x, y)
# 三
z = torch.empty(5, 3)
torch.add(x, y, out=z) # 将结果放在z中
# 四
y.add_(x) # 原地操作：y = x + y

tensor([[1.2114, 0.4820, 0.4608],
        [0.3709, 1.1454, 1.4792],
        [0.7677, 1.4011, 0.5814],
        [1.3993, 0.2273, 0.6034],
        [1.1362, 0.6134, 1.2738]])

注意：
* 在任何一个in-place改变张量的操作后面都固定一个'_'（例如x.copy_(y)、x.t_()），将原地更改x
* 可以使用像标准的NumPy一样的各种索引操作（print(x\[:, 1\])）

## 形状
改变形状：如果想改变形状，可以使用torch.view()

In [7]:
x = torch.randn(4, 4) # 生成数值取自均值为0，方差为1的正态分布（标准正态分布）
y = x.view(16)
z = x.view(-1, 8) # -1表示该值取决于其他维度
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [8]:
x = torch.randn(1)
print(x)
print(x.item()) # 单个元素可以用.item()表示它的值

tensor([-1.1311])
-1.131130576133728


关于tensor的所有函数请看[这里](https://pytorch.org/docs/stable/torch.html)

## Numpy与Torch的转换
Torch张量和NumPy数组将共享它们的底层内存位置，因此当一个改变时,另外也会改变。

In [9]:
a = torch.ones(2, 2)
a

tensor([[1., 1.],
        [1., 1.]])

In [10]:
b = a.numpy()  # 从tensor到numpy数组
b              # 此时a与b指代同一个东西

array([[1., 1.],
       [1., 1.]], dtype=float32)

In [12]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a) # 从numpy到tensor
np.add(a, 1, out=a)     # 同时改变
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


Cuda上的tensor可以用.to()方法移动到任何设备上

In [13]:
# 当GPU可用时,我们可以运行以下代码
# 我们将使用`torch.device`来将tensor移入和移出GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # 直接在GPU上创建tensor
    x = x.to(device)                       # 或者使用`.to("cuda")`方法
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # `.to`也能在移动时改变dtype

tensor([-0.1311], device='cuda:0')
tensor([-0.1311], dtype=torch.float64)


## Autograd：自动求导
来源于<https://pytorch.apachecn.org/docs/1.4/blitz/autograd_tutorial.html>  
PyTorch中，所有神经网络的核心是 autograd 包。先简单介绍一下这个包，然后训练我们的第一个的神经网络。

autograd 包为张量上的所有操作提供了自动求导机制。它是一个在运行时定义(define-by-run）的框架，这意味着反向传播是根据代码如何运行来决定的，并且每次迭代可以是不同的。
### 追踪

In [14]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
y = x + 2 # 广播
print(y)  # 追踪的操作放在grad_fn中

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [15]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


### 梯度

In [17]:
out.backward()

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [18]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


## 梯度下降、反向传播、雅可比矩阵
[梯度下降](https://blog.csdn.net/qq_41800366/article/details/86583789)、[反向传播](https://blog.csdn.net/weixin_38347387/article/details/82936585)、[雅可比矩阵](https://www.cnblogs.com/rswss/p/11440144.html)